In [25]:
import json  
  
def read_jsonl(file_path):  
    """  
    读取 JSONL 文件中的数据并返回一个包含所有记录的字典列表。  
  
    参数:  
    file_path (str): JSONL 文件的路径。  
  
    返回:  
    list: 包含所有记录的字典列表。  
    """  
    data = []  
  
    with open(file_path, 'r', encoding='utf-8') as file:  
        for line in file:  
            # 解析每一行的 JSON 对象并添加到列表中  
            data.append(json.loads(line.strip()))  
  
    return data  
model_name = 'ta_chosen_tuluv2_dpo_2048_default_template'
file_path = f'data/arena-hard-v0.1/model_answer/{model_name}.jsonl'
data = read_jsonl(file_path)
judge_data_path = f'data/arena-hard-v0.1/model_judgment/gpt-4o/{model_name}.jsonl'
judge_data = read_jsonl(judge_data_path)
# from utils import read_jsonl  
import os
import concurrent.futures  
from text2img import text_to_image  
def process_item(item):  
    index = item["question_id"]  
    output_img_original = f"output_original_{index}.png"  
    # print(item['choices'][0]['turns'][0]['content'])
    text_to_image(item['choices'][0]['turns'][0]['content'], output_img_original, save_dir="original_response", temp_dir="original_temp")  
    # text_to_image("here", output_img_original, save_dir="original_response", temp_dir="original_temp")  
    print("here")
    # output_img = f"output_{index}.png"  
    # text_to_image(item["gpt_answer"], output_img, save_dir="gpt_response", temp_dir="gpt_temp")  
    # output_img = f"output_{index}.png"  
    # text_to_image(item["revised_text"], output_img, save_dir="revised_response", temp_dir="revised_temp")  
def process_judge(item):  
    index = item["question_id"]  
    output_img_original = f"output_original_{index}.png"  
    judge_data[0]['games'][0]['user_prompt'], judge_data[0]['games'][0]['judgment']
    # print(item['choices'][0]['turns'][0]['content'])
    text_to_image(item['games'][0]['user_prompt'], output_img_original, save_dir="user_prompt", temp_dir="user_prompt_temp")  
    text_to_image(item['games'][0]['judgment'], output_img_original, save_dir="judge_response", temp_dir="judge_temp")  
    # text_to_image("here", output_img_original, save_dir="original_response", temp_dir="original_temp")  
    # print("here")

def generate_res(data, judge_data):
    # cur_dir = os.path.dirname(os.path.abspath(__file__))
    # data_path = os.path.join(cur_dir, "revised_data/output_sorted.jsonl")
    # revised_data = read_jsonl(data_path)  
    
    # 指定线程数，例如 4  
    max_workers = 20  
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:  
        futures = [executor.submit(process_item, item) for item in data[:100]]  
        for future in concurrent.futures.as_completed(futures):  
            try:  
                future.result()  
            except Exception as exc:  
                print(f'Generated an exception: {exc}')  
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:  
        futures = [executor.submit(process_judge, item) for item in judge_data[:100]]  
        for future in concurrent.futures.as_completed(futures):  
            try:  
                future.result()  
            except Exception as exc:  
                print(f'Generated an exception: {exc}')  
generate_res(data, judge_data)

2024-08-30 04:31:22,024 - INFO - Starting the conversion process.
2024-08-30 04:31:22,025 - INFO - Starting the conversion process.
2024-08-30 04:31:22,026 - INFO - Starting the conversion process.
2024-08-30 04:31:22,026 - INFO - Starting the conversion process.
2024-08-30 04:31:22,028 - INFO - Starting the conversion process.
2024-08-30 04:31:22,029 - INFO - Starting the conversion process.
2024-08-30 04:31:22,032 - INFO - Starting the conversion process.
2024-08-30 04:31:22,034 - INFO - Starting the conversion process.
2024-08-30 04:31:22,034 - INFO - Starting the conversion process.
2024-08-30 04:31:22,038 - INFO - Starting the conversion process.
2024-08-30 04:31:22,039 - INFO - Starting the conversion process.
[WA2024-08-30 04:31:22,043 - INFO - Starting the conversion process.
2024-08-30 04:31:22,050 - INFO - Starting the conversion process.
RNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by 

output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
Generated an exception: [Errno 17] File exists: '/home/lidong1/jianglingjie/arena-hard-auto/output/original_response'
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_

[WARNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling back to 'output_original_09e03665fa144757920dbf7d8946ad30'
[WARNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling back to 'output_original_089cea4c41974cfa9ba05694c87c5685'
2024-08-30 04:31:22,284 - INFO - 转换成功: /home/lidong1/jianglingjie/arena-hard-auto/output/original_temp/output_original_09e03665fa144757920dbf7d8946ad30.html
2024-08-30 04:31:22,302 - INFO - 转换成功: /home/lidong1/jianglingjie/arena-hard-auto/output/original_temp/output_original_089cea4c41974cfa9ba05694c87c5685.html
/snap/chromium/2934/gnome-platform/command-chain/desktop-launch: line 316: /dev/null: Operation not permitted
[3679524:3680377:0830/043122.549978:ERROR:obje

[3679599:3681758:0830/043122.853688:ERROR:object_proxy.cc(576)] Failed to call method: org.freedesktop.DBus.ListActivatableNames: object_path= /org/freedesktop/DBus: org.freedesktop.DBus.Error.AccessDenied: An AppArmor policy prevents this sender from sending this message to this recipient; type="method_call", sender=":1.20209" (uid=1003 pid=3679599 comm="/snap/chromium/2934/usr/lib/chromium-browser/chrom" label="snap.chromium.chromium (enforce)") interface="org.freedesktop.DBus" member="ListActivatableNames" error name="(unset)" requested_reply="0" destination="org.freedesktop.DBus" (bus)
[3679674:3682169:0830/043122.856157:ERROR:object_proxy.cc(576)] Failed to call method: org.freedesktop.DBus.ListActivatableNames: object_path= /org/freedesktop/DBus: org.freedesktop.DBus.Error.AccessDenied: An AppArmor policy prevents this sender from sending this message to this recipient; type="method_call", sender=":1.20211" (uid=1003 pid=3679674 comm="/snap/chromium/2934/usr/lib/chromium-browser/

here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response


...

[WARNING] This document format requires a non118518emp bytes written to file t/home/lidong1/jianglingjie/arena-hard-auto/output/user_prompt/output_original_4a0cf903c61b47ceb49166b591c703d5.pngy
 <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling back to 'outp181927ut bytes written to file _/home/lidong1/jianglingjie/arena-hard-auto/output/user_prompt/output_original_01b5156495464638b98e1f8d9be12c23.pngo
riginal_3801e82e67a34ee3aaab0227342bcc2b'
[WARNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling back to 'output_original_0b70bc948cda4825b0a18438020fe1db'
2024-08-30 04:31:39,575 - INFO - Image saved to /home/lidong1/jianglingjie/arena-hard-auto/output/user_prompt/output_original_40b8576cb60b42619b0c5530c20ea457.png.
114025 bytes writt

output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/judge_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/judge_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/judge_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/judge_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/judge_response


[3749450:3750913:0830/043139.789758:ERROR:object_proxy.cc(576)] Failed to call method: org.freedesktop.DBus.ListActivatableNames: object_path= /org/freedesktop/DBus: org.freedesktop.DBus.Error.AccessDenied: An AppArmor policy prevents this sender from sending this message to this recipient; type="method_call", sender=":1.21849" (uid=1003 pid=3749450 comm="/snap/chromium/2934/usr/lib/chromium-browser/chrom" label="snap.chromium.chromium (enforce)") interface="org.freedesktop.DBus" member="ListActivatableNames" error name="(unset)" requested_reply="0" destination="org.freedesktop.DBus" (bus)
2024-08-30 04:31:39,797 - INFO - 转换成功: /home/lidong1/jianglingjie/arena-hard-auto/output/judge_temp/output_original_dc7f0ea67aa94095adc8b48205af363b.html
[3749617:3751138:0830/043139.831780:ERROR:object_proxy.cc(576)] Failed to call method: org.freedesktop.DBus.ListActivatableNames: object_path= /org/freedesktop/DBus: org.freedesktop.DBus.Error.AccessDenied: An AppArmor policy prevents this sender fr

output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/judge_response


[3750279:3752369:0830/043140.072740:ERROR:object_proxy.cc(576)] Failed to call method: org.freedesktop.DBus.ListActivatableNames: object_path= /org/freedesktop/DBus: org.freedesktop.DBus.Error.AccessDenied: An AppArmor policy prevents this sender from sending this message to this recipient; type="method_call", sender=":1.21883" (uid=1003 pid=3750279 comm="/snap/chromium/2934/usr/lib/chromium-browser/chrom" label="snap.chromium.chromium (enforce)") interface="org.freedesktop.DBus" member="ListActivatableNames" error name="(unset)" requested_reply="0" destination="org.freedesktop.DBus" (bus)
[3750221:3752129:0830/043140.085552:ERROR:object_proxy.cc(576)] Failed to call method: org.freedesktop.DBus.ListActivatableNames: object_path= /org/freedesktop/DBus: org.freedesktop.DBus.Error.AccessDenied: An AppArmor policy prevents this sender from sending this message to this recipient; type="method_call", sender=":1.21885" (uid=1003 pid=3750221 comm="/snap/chromium/2934/usr/lib/chromium-browser/

In [13]:
data[0]['choices'][0]['turns'][0]['content']

'**Implementing a SCIM Server in PHP using Slim**\n\n**Overview**\n\nThis guide will walk you through implementing a SCIM (System for Cross-domain Identity Management) server in PHP using the Slim framework as a router.\n\n**Prerequisites**\n\n- PHP 7.4 or higher\n- Slim Framework 4\n- Composer\n\n**Step 1: Set up the Project**\n\nCreate a new PHP project and install the required dependencies using Composer:\n\n```bash\ncomposer require slim/slim slim/slim-routing\n```\n\n**Step 2: Define the SCIM API Endpoints**\n\nThe SCIM API has several endpoints for managing users, groups, and other resources. Here are the endpoints you\'ll need to implement:\n\n- **Users**\n  - `GET /Users`: Retrieve a list of users\n  - `GET /Users/{id}`: Retrieve a specific user\n  - `POST /Users`: Create a new user\n  - `PATCH /Users/{id}`: Update an existing user\n  - `DELETE /Users/{id}`: Delete a user\n\n- **Groups**\n  - `GET /Groups`: Retrieve a list of groups\n  - `GET /Groups/{id}`: Retrieve a specific 

In [8]:
data[0]

{'question_id': '0122ab60646b4961bc39e9c03bdf6bcc',
 'answer_id': 'Xy5QRb7kGE8XrzNyPaZqbZ',
 'model_id': 'ta_rejected_llama3.1_instruct_dpo_2048',
 'choices': [{'index': 0,
   'turns': [{'content': '**Implementing a SCIM Server in PHP using Slim**\n\n**Overview**\n\nThis guide will walk you through implementing a SCIM (System for Cross-domain Identity Management) server in PHP using the Slim framework as a router.\n\n**Prerequisites**\n\n- PHP 7.4 or higher\n- Slim Framework 4\n- Composer\n\n**Step 1: Set up the Project**\n\nCreate a new PHP project and install the required dependencies using Composer:\n\n```bash\ncomposer require slim/slim slim/slim-routing\n```\n\n**Step 2: Define the SCIM API Endpoints**\n\nThe SCIM API has several endpoints for managing users, groups, and other resources. Here are the endpoints you\'ll need to implement:\n\n- **Users**\n  - `GET /Users`: Retrieve a list of users\n  - `GET /Users/{id}`: Retrieve a specific user\n  - `POST /Users`: Create a new user\

In [15]:
# from utils import read_jsonl  
import os
import concurrent.futures  
from text2img import text_to_image  
def process_item(item):  
    index = item["question_id"]  
    output_img_original = f"output_original_{index}.png"  
    # print(item['choices'][0]['turns'][0]['content'])
    text_to_image(item['choices'][0]['turns'][0]['content'], output_img_original, save_dir="original_response", temp_dir="original_temp")  
    # text_to_image("here", output_img_original, save_dir="original_response", temp_dir="original_temp")  
    print("here")
    # output_img = f"output_{index}.png"  
    # text_to_image(item["gpt_answer"], output_img, save_dir="gpt_response", temp_dir="gpt_temp")  
    # output_img = f"output_{index}.png"  
    # text_to_image(item["revised_text"], output_img, save_dir="revised_response", temp_dir="revised_temp")  

def generate_res(data):
    # cur_dir = os.path.dirname(os.path.abspath(__file__))
    # data_path = os.path.join(cur_dir, "revised_data/output_sorted.jsonl")
    # revised_data = read_jsonl(data_path)  
    
    # 指定线程数，例如 4  
    max_workers = 20  
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:  
        futures = [executor.submit(process_item, item) for item in data[:2000]]  
        for future in concurrent.futures.as_completed(futures):  
            try:  
                future.result()  
            except Exception as exc:  
                print(f'Generated an exception: {exc}')  
generate_res(data)

2024-08-30 01:20:00,406 - INFO - Starting the conversion process.
2024-08-30 01:20:00,408 - INFO - Starting the conversion process.
2024-08-30 01:20:00,410 - INFO - Starting the conversion process.
2024-08-30 01:20:00,414 - INFO - Starting the conversion process.
2024-08-30 01:20:00,417 - INFO - Starting the conversion process.
2024-08-30 01:20:00,421 - INFO - Starting the conversion process.
2024-08-30 01:20:00,422 - INFO - Starting the conversion process.
2024-08-30 01:20:00,428 - INFO - Starting the conversion process.
2024-08-30 01:20:00,429 - INFO - Starting the conversion process.
2024-08-30 01:20:00,430 - INFO - Starting the conversion process.
2024-08-30 01:20:00,439 - INFO - Starting the conversion process.
[WARN2024-08-30 01:20:00,443 - INFO - Starting the conversion process.
ING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling ba

output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/j

[WARNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling back to 'output_original_03917c9052f0457c97e6114e810747a9'
[WARNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling back to 'output_original_09e03665fa144757920dbf7d8946ad30'
2024-08-30 01:20:00,643 - INFO - 转换成功: /home/lidong1/jianglingjie/arena-hard-auto/output/original_temp/output_original_03917c9052f0457c97e6114e810747a9.html
2024-08-30 01:20:00,663 - INFO - 转换成功: /home/lidong1/jianglingjie/arena-hard-auto/output/original_temp/output_original_09e03665fa144757920dbf7d8946ad30.html
[WARNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metada

here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
here
here
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response
output_path:L/home/lidong1/jianglingjie/arena-hard-auto/output/original_response


126598 bytes written to file /home/lidong1/jianglingjie/arena-hard-auto/output/original_response/output_original_06991f6da3104bf8b935d311581a348e.png
[WARNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling back to 'output_original_0dea89391d074b73a19c8e48ece8640c'
96801 bytes written to file /home/lidong1/jianglingjie/arena-hard-auto/output/original_response/output_original_08d2f41143ff411c83c02a521eb01948.png
[WARNING] This document format requires a nonempty <title> element.
  Please specify either 'title' or 'pagetitle' in the metadata,
  e.g. by using --metadata pagetitle="..." on the command line.
  Falling back to 'output_original_0e07d745af7e4ec9a2769b77e7ae8ca7'
2024-08-30 01:20:01,804 - INFO - 转换成功: /home/lidong1/jianglingjie/arena-hard-auto/output/original_temp/output_original_0dea89391d074b73a19c8e48ece8640c.html
2024-08-30 01:

...

here
here
here
here
here


171728 bytes written to file /home/lidong1/jianglingjie/arena-hard-auto/output/original_response/output_original_fdfea302ee4246689541d7e93e7774a2.png
2024-08-30 01:20:30,019 - INFO - Image saved to /home/lidong1/jianglingjie/arena-hard-auto/output/original_response/output_original_fb3496a54f514e729c98f36bd6058ffd.png.
2024-08-30 01:20:30,020 - INFO - Conversion process completed successfully.
2024-08-30 01:20:30,029 - INFO - Image saved to /home/lidong1/jianglingjie/arena-hard-auto/output/original_response/output_original_fd6e7856e1fe489086a831a7893ac3fa.png.
2024-08-30 01:20:30,029 - INFO - Conversion process completed successfully.
2024-08-30 01:20:30,047 - INFO - Image saved to /home/lidong1/jianglingjie/arena-hard-auto/output/original_response/output_original_fd86dff7ac084b99889d294e135f82a4.png.
2024-08-30 01:20:30,048 - INFO - Conversion process completed successfully.
93227 bytes written to file /home/lidong1/jianglingjie/arena-hard-auto/output/original_response/output_original_

here
here
here
here
here
here


2024-08-30 01:20:30,290 - INFO - Image saved to /home/lidong1/jianglingjie/arena-hard-auto/output/original_response/output_original_ffb30214c4d245ec874e292a0490445c.png.
2024-08-30 01:20:30,290 - INFO - Conversion process completed successfully.


here


In [23]:
judge_data = read_jsonl("data/arena-hard-v0.1/model_judgment/gpt-4o/ta_chosen_tuluv2_dpo_2048_default_template.jsonl")
judge_data[0]['games'][0]['user_prompt'], judge_data[0]['games'][0]['judgment']

("<|User Prompt|>\nDesign a semikinematic mounting for a right angle prism with preload provided by a compressed elastomeric pad. The mounting should be designed to ensure proper alignment of the prism with its mounting surface and provide adequate tension to maintain proper load transfer.\n\n<|The Start of Assistant A's Answer|>\nTo design a semikinematic mounting for a right angle prism with preload provided by a compressed elastomeric pad, follow these steps:\n\n1. Select the right angle prism: Choose a right angle prism with the desired dimensions and optical properties for your application. The prism should have a high-quality surface finish and be made of a suitable material, such as BK7 glass or fused silica.\n\n2. Design the mounting base: Create a mounting base that will securely hold the right angle prism. The base should have a flat surface for the prism to rest on, with a groove or recess to accommodate the elastomeric pad. The groove should be slightly shallower than the t

In [29]:
judge_data = read_jsonl("data/arena-hard-v0.1/model_judgment/gpt-4o/ta_chosen_tuluv2_dpo_2048_default_template.jsonl")
judge_data[0]['games'][0]['user_prompt'], judge_data[0]['games'][0]['judgment']
judges = [item['games'][0]['score'] for item in judge_data]
judges

['A>>B',
 'A>>B',
 'A>>B',
 'B>A',
 'A>>B',
 'A>>B',
 'B>A',
 'A>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'B>A',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'B>A',
 'B>A',
 'A>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>B',
 'A>>B',
 'A>B',
 'A>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>>B',
 None,
 'A>>B',
 'B>A',
 'A>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>B',
 'A>>B',
 'A>>B',
 'B>A',
 'A>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'B>A',
 'A>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'B>A',
 'A>B',
 'B>A',
 'A>B',
 'A>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>>B',
 'A>>B',
 'A>B',
 'A>>B',
 'A>B',
 '